In [2]:
import sys
from pathlib import Path
PARENT_DIR = Path.cwd().parent.parent
sys.path.append(str(PARENT_DIR))

In [11]:
list((PARENT_DIR / "data/trainingset-ideology-power").glob("*"))

[WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/.gitignore'),
 WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/.ipynb_checkpoints'),
 WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/.vector_cache'),
 WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/Bag-of-word_binary_vector.ipynb'),
 WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/basic_neural_network.py'),
 WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/basic_nn_pipeline.py'),
 WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/contribution.txt'),
 WindowsPath('C:/Users/72786/Documents/nodalida/power-identification/data/trainingset-ideology-power/data'),
 WindowsPath('C:/Users/72

In [3]:
# Get the last path in sys.path
last_path = Path(sys.path[-1])

# Check if last_path is a valid directory
if last_path.exists() and last_path.is_dir():
    # Iterate over all directories in the last path and add them to sys.path
    for item in last_path.iterdir():
        if item.is_dir():
            sys.path.append(str(item))  # Add each directory to sys.path
else:
    print(f"{last_path} is not a valid directory")

# Verify that folders have been added to sys.path
sys.path

['C:\\Users\\72786\\Documents\\nodalida\\power-identification\\experiments\\feature_engineering',
 'C:\\Users\\72786\\anaconda3\\python311.zip',
 'C:\\Users\\72786\\anaconda3\\DLLs',
 'C:\\Users\\72786\\anaconda3\\Lib',
 'C:\\Users\\72786\\anaconda3',
 '',
 'C:\\Users\\72786\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\72786\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\72786\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\72786\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\72786\\Documents\\nodalida\\power-identification',
 'C:\\Users\\72786\\Documents\\nodalida\\power-identification\\.git',
 'C:\\Users\\72786\\Documents\\nodalida\\power-identification\\archived',
 'C:\\Users\\72786\\Documents\\nodalida\\power-identification\\data',
 'C:\\Users\\72786\\Documents\\nodalida\\power-identification\\experiments',
 'C:\\Users\\72786\\Documents\\nodalida\\power-identification\\lib']

In [6]:
from datasets import load_dataset

# Load the dataset from the TSV file
dataset = load_dataset('csv', data_files=PARENT_DIR\power-gb-train.tsv", delimiter='\t')

# Split the dataset into train and test sets (e.g., 80-20 split)
split_dataset = dataset['train'].train_test_split(test_size=0.2)


In [3]:
# Extract train and test data
train_texts = split_dataset['train']['text']
train_labels = split_dataset['train']['label']
test_texts = split_dataset['test']['text']
test_labels = split_dataset['test']['label']


In [9]:
from typing import Optional

from torch.utils.data import DataLoader, TensorDataset
import torch
from torch import nn, optim
from tqdm import tqdm

import numpy as np

from lib.data_processing import PositionalEncoder

In [10]:
from models import TrainConfig, NeuralNetwork, save_model, load_model

NameError: name 'DataLoader' is not defined

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class BasicNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BasicNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Helper function to train and evaluate the neural network
def train_and_evaluate(train_vectors, train_labels, test_vectors, test_labels, input_size, hidden_size=50, num_classes=2, epochs=10, batch_size=16):
    # Convert data to PyTorch tensors and move to GPU
    train_vectors = torch.tensor(train_vectors, dtype=torch.float32).to(device)
    train_labels = torch.tensor(train_labels, dtype=torch.long).to(device)
    test_vectors = torch.tensor(test_vectors, dtype=torch.float32).to(device)
    test_labels = torch.tensor(test_labels, dtype=torch.long).to(device)

    # Create data loaders
    train_dataset = TensorDataset(train_vectors, train_labels)
    test_dataset = TensorDataset(test_vectors, test_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, loss function, and optimizer
    model = BasicNN(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(epochs):
        model.train()
        for vectors, labels in train_loader:
            outputs = model(vectors)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for vectors, labels in test_loader:
            outputs = model(vectors)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='weighted')
    return f1


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

bow_tfidf_maxfeatures = [1000, 5000, 10000, 20000, 50000]
for i in bow_tfidf_maxfeatures:
    # Bag-of-Words with max_features
    bow_vectorizer = CountVectorizer(max_features=i)
    train_vectors = bow_vectorizer.fit_transform(train_texts).toarray()
    test_vectors = bow_vectorizer.transform(test_texts).toarray()
    bow_f1 = train_and_evaluate(train_vectors, train_labels, test_vectors, test_labels, input_size=i)
    print(f'Bag-of-Words F1 Score: {bow_f1}')

    # TF-IDF with max_features
    tfidf_vectorizer = TfidfVectorizer(max_features=i)
    train_vectors = tfidf_vectorizer.fit_transform(train_texts).toarray()
    test_vectors = tfidf_vectorizer.transform(test_texts).toarray()
    tfidf_f1 = train_and_evaluate(train_vectors, train_labels, test_vectors, test_labels, input_size=i)
    print(f'TF-IDF F1 Score: {tfidf_f1}')


Bag-of-Words F1 Score: 0.7554709447215134
TF-IDF F1 Score: 0.7409388745905656


In [30]:
import numpy as np

# Load GloVe embeddings
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Example GloVe file path (adjust as needed)
glove_file = './.vector_cache/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file)

# Tokenize and vectorize using GloVe
glove_dims = [100, 200, 300]
def glove_vectorize(texts, embeddings, embedding_dim):
    vectors = []
    for text in texts:
        words = text.split()
        word_vectors = [embeddings.get(word, np.zeros(embedding_dim)) for word in words]
        if word_vectors:
            vectors.append(np.mean(word_vectors, axis=0))
        else:
            vectors.append(np.zeros(embedding_dim))
    return np.array(vectors)

for i in golve_dims:
    train_vectors = glove_vectorize(train_texts, glove_embeddings, i)
    test_vectors = glove_vectorize(test_texts, glove_embeddings, i)
    glove_f1 = train_and_evaluate(train_vectors, train_labels, test_vectors, test_labels, input_size=i)
    print(f'GloVe F1 Score: {glove_f1}')


GloVe F1 Score: 0.7009512004441185
